In [ ]:
from zipfile import ZipFile

f = '/content/Humor,Hist,Media,Food.zip'

with ZipFile(f,'r') as zip:
  zip.printdir()
  zip.extractall()

In [79]:
import numpy as np
import pandas as pd


In [80]:
df = pd.read_csv("/content/Humor,Hist,Media,Food/a-team",sep='\t',names=["text"])
df

,text
0,From uunet!cs.utexas.edu!usc!ucsd!ucbvax!CAE.W...
1,Article 23580 of rec.arts.tv:
2,Path: microsoft!uunet!cs.utexas.edu!usc!ucsd!u...
3,From: russellp@CAE.WISC.EDU
4,Newsgroups: rec.arts.tv
...,...
498,*************** russellp@cae.wisc.edu ********...
499,"Russ Perry Jr russell@uhura.cs.wisc.edu ""..."
500,5970 Scott St 104 S Randall Ave ...
501,Omro WI 54963 Madison WI 53715 that ...


In [81]:
df['text'][0]

'From uunet!cs.utexas.edu!usc!ucsd!ucbvax!CAE.WISC.EDU!russellp Wed May 23 13:07:08 PDT 1990'

In [82]:
t = df["text"].values


Reading the files

In [ ]:
with open('/content/Humor,Hist,Media,Food/calvin.txt',encoding='utf8',errors='ignore') as f:
    lines = f.readlines()

lines

Inverted Index function

In [84]:
def inverted_index(data:list):
  dict1 = {}
  for ind,text in enumerate(data):
    for w in text.split():
      if w not in dict1.keys():
        dict1[w] = [ind]
      else:
        dict1[w].append(ind)

  return dict1




In [85]:
import os
p = "/content/Humor,Hist,Media,Food"
name = os.listdir(p)


In [86]:
name = sorted(name)

In [87]:
name[1]

'a-team'

In [88]:
doc = []

for i in range(1,len(name)):
  # print(name[i])
  # print(i)
  
  # if '.' not in l[i]:
  #   l[i]+=".txt"
  try:
    with open("/content/Humor,Hist,Media,Food/"+name[i],encoding='utf8',errors='ignore') as f:
      lines = f.readlines()
    t1 = " ".join(lines)
    doc.append(t1)
  except:
    df = pd.read_csv("/content/Humor,Hist,Media,Food/"+name[i],sep='\t',lineterminator='\n',names=["text"])
    df.dropna(inplace=True)
    t = df["text"].values
    # print(t)
    t1 = " ".join(t)
    doc.append(t1)


In [89]:
len(doc)

1132

In [90]:
df = pd.read_csv("/content/Humor,Hist,Media,Food/"+name[117],sep='\t',lineterminator='\n',names=["text"])


In [92]:
import re
import nltk
import string

In [93]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [94]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [95]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unicodedata

Function for removing stopwords

In [96]:
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return " ".join(filtered_text)

Function for removing non ascii words

In [97]:
def non_ascii_removal(words):
    """Remove non-ASCII characters from list of tokenized words"""
    words = word_tokenize(words)
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return " ".join(new_words)

In [98]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [99]:
nltk.pos_tag(["dogs"])

[('dogs', 'NNS')]

In [100]:
from nltk.stem import WordNetLemmatizer
 
lemmatizer = WordNetLemmatizer()

In [101]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [102]:
lemmatizer.lemmatize("giris", pos ='v')

'giris'

In [103]:
def lemmatize_word(text):
    word_tokens = word_tokenize(text)
    # provide context i.e. part-of-speech
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens]
    return " ".join(lemmas)

In [104]:
remove_stopwords('the air is big')

'air big'

Preprocessing the files

In [105]:
for i in range(len(doc)):
  doc[i] = doc[i].lower()
  
  doc[i] = re.sub(r'[^\w\s]', ' ', doc[i])
  
  doc[i] = " ".join(doc[i].split())
  doc[i] = remove_stopwords(doc[i])



In [106]:
doc[1]

'murph buscard fidonet org brian murphy subject fish called wanda apology wanted date thu 24 jun 93 03 49 47 edt tuned group knew place ask words apology offered john cleese hanging upside high window held kevin klein movie fish called wanda sure enough day two later gent australia answered thought share date thu 24 jun 93 10 55 16 est yallara cs rmit oz au s933384 dave cole subject fish called wanda apology wanted g day brian apology scene fish called wanda script otto take back camera turning 180 degrees pulls back rapidly reveal archie dangling upside window flat otto holding ankles river bed uncovered water low tide seventy feet archie offer complete utter retraction imputation totally without basis fact way fair comment motivated purely malice deeply regret distress comments may caused family people bank staring hereby undertake repeat slander time future otto hesitates ok thanks dave sure appreciate believe going try funny business girlfriend weekend knowing apology come handy gr

In [107]:
len(doc)

1132

Forming the inverted index

In [108]:
d = inverted_index(doc)

Making the document ids unique

In [109]:
import numpy as np
for key,value in enumerate(d):
  d[value] = np.unique(d[value])

making the document ids sorted

In [110]:
for key,value in enumerate(d):
  sorted(d[value])
  

In [111]:
fq ={}

Finding the length of each posting

In [112]:
for key,value in enumerate(d):
  
  fq[value] = len(d[value])

In [113]:
d['betsy']

array([   0,   57,  397,  425,  449,  662,  751, 1036, 1053])

Getting the inputs

In [114]:
s1 = input("Input query: ")
s2 = input("Input operation sequence:")

Input query: please let be
Input operation sequence:[or, or]


In [115]:
s1

'please let be'

Preprocessing the inputs

In [116]:

s3 = "".join(s2.split(" "))

In [117]:
s3

'[or,or]'

In [118]:
if s3[0]=='[':
  s3  = s3[1:len(s3)-1]
s3 = s3.lower()

In [119]:
s3

'or,or'

In [120]:
s3 = s3.split(",")
s3

['or', 'or']

In [121]:
s1 = s1.lower()
s1 = re.sub(r'[^\w\s]', ' ', s1)
s1 = " ".join(s1.split())



In [122]:
s1 = remove_stopwords(s1)
s1

'please let'

In [123]:
s1 = s1.split()
s1

['please', 'let']

In [124]:
s3 = s3[:len(s1)-1]

In [125]:
def comp(a):
  return a[1]

In [126]:
dq = {}
for q in s1:
  dq[q] = fq[q]

dq =sorted(dq,key=comp)
dq

['let', 'please']

In [127]:
dq

['let', 'please']

In [128]:
s1

['please', 'let']

In [129]:

dq1 = {}
for q in dq:
  dq1[q] = fq[q]
dq1

{'let': 411, 'please': 264}

Function of each operations

In [130]:
def OR(x,y):
  a=0
  b=0
  r = []
  count=0
  while a < len(x) and b < len(y):
    count+=1
    if x[a] == y[b]:
      r.append(x[a])
      a+=1
      b+=1
    elif x[a] > y[b]:
      r.append(y[b])
      b+=1
    else:
      r.append(x[a])
      a+=1
  while a<len(x):
    r.append(x[a])
    a+=1

  while b<len(y):
    r.append(y[b])
    b+=1

  return len(r),count,r

In [131]:
def AND(x, y):
    a = 0
    b = 0
    r = []
    count=0
    while a < len(x) and b < len(y):
        count+=1
        if x[a] == y[b]:
            r.append(x[a])
            a += 1
            b += 1
        elif x[a] > y[b]:
            b += 1
        else:
            a += 1
    return len(r),count,r

In [132]:
def NOT(x):
    r = []
    i = 0
    for j in x:
        while i < j:
            r.append(i)
            i+=1
        i += 1
        
    while i < len(doc):
        r.append(i)
        i += 1
    return r

In [133]:
def ANDNOT(x,y):
  temp1 = NOT(y)
  l3,c3,temp2 = AND(x,temp1)
  return  l3,c3,temp2


In [134]:
def ORNOT(x,y):
  temp1 = NOT(y)
  l3,c3,temp2 = OR(x,temp1)
  return  l3,c3,temp2

Performing the query

In [135]:
c=0
l=0
temp=d[s1[0]]
for i in range(1,len(s1)):
  if(s3[i-1]=='or'):
    l1,c1,t = OR(temp,d[s1[i]])
    # print(c1)
    c+=c1
    l+=l1
    temp = t.copy()
  elif(s3[i-1]=="and"):
    l1,c1,t = AND(temp,d[s1[i]])
    # print(c1)
    c+=c1
    l+=l1
    temp = t.copy()
  elif(s3[i-1]=='ornot'):
    l1,c1,t = ORNOT(temp,d[s1[i]])
    # print(c1)
    c+=c1
    l+=l1
    temp = t.copy()
  elif(s3[i-1]=='andnot'):
    l1,c1,t = ANDNOT(temp,d[s1[i]])
    # print(c1)
    c+=c1
    l+=l1
    temp = t.copy()

print("No. of comparisions : " + str(c))
print("No. of documents matched : "+str(l))
print(temp)

No. of comparisions : 516
No. of documents matched : 517
[0, 3, 4, 8, 10, 15, 16, 18, 19, 22, 27, 28, 29, 30, 33, 35, 37, 39, 41, 42, 44, 45, 47, 48, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 66, 82, 83, 87, 89, 90, 91, 92, 94, 96, 98, 99, 104, 105, 107, 111, 115, 116, 117, 118, 119, 122, 125, 126, 127, 128, 129, 130, 131, 134, 139, 144, 146, 149, 150, 153, 155, 163, 167, 172, 173, 174, 177, 179, 181, 182, 183, 185, 186, 188, 193, 194, 197, 198, 201, 202, 205, 207, 208, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 223, 227, 231, 232, 233, 235, 238, 240, 243, 244, 245, 247, 253, 254, 255, 256, 263, 266, 267, 268, 269, 271, 272, 274, 275, 277, 281, 283, 284, 285, 286, 287, 293, 297, 298, 304, 305, 306, 307, 308, 313, 314, 315, 317, 318, 320, 323, 327, 330, 341, 342, 343, 344, 345, 346, 347, 348, 350, 352, 353, 354, 359, 360, 362, 366, 367, 369, 377, 378, 379, 390, 391, 393, 404, 409, 412, 413, 417, 418, 420, 423, 425, 427, 430, 431, 433, 436, 440, 442, 443, 444, 447, 448, 449, 450, 45

In [136]:
len(d['stand'])

186

In [137]:
len(d)

71249

# **Positional** **Index**

In [138]:
doc10 = []

for i in range(1,len(name)):
  
  # if '.' not in l[i]:
  #   l[i]+=".txt"
  try:
    with open("/content/Humor,Hist,Media,Food/"+name[i],encoding='utf8',errors='ignore') as f:
      lines = f.readlines()
    t1 = " ".join(lines)
    doc10.append(t1)
  except:
    df = pd.read_csv("/content/Humor,Hist,Media,Food/"+name[i],sep='\t',lineterminator='\n',names=["text"])
    df.dropna(inplace=True)
    t = df["text"].values
    # print(t)
    t1 = " ".join(t)
    doc10.append(t1)

In [139]:
def remove_stopwords1(text):
    stop_words = set(stopwords.words("english"))
    # word_tokens = word_tokenize(text)
    filtered_text = [word for word in text if word not in stop_words]
    return filtered_text

Preprocessing the document

In [140]:
from string import punctuation
for i in range(len(doc10)):
  doc10[i] = doc10[i].lower()
  
  doc10[i] = re.sub(f"[{re.escape(punctuation)}]", "", doc10[i])
  
  doc10[i] = " ".join(doc10[i].split())
  
  doc10[i] = remove_stopwords(doc10[i])
  doc10[i] = word_tokenize(doc10[i])
  

In [141]:
len(doc10)

1132

Function for the positional inverted index

In [142]:
def positional_inverted_index(data:list):
  dict1 = {}
  for ind,text in enumerate(data):
    for ind1,w in enumerate(text):
      if w not in dict1.keys():
        
        d7 = [ind1]
        dict1[w] = [[ind,d7]]
      else:
        count=0
        for i in range(len(dict1[w])):
          if dict1[w][i][0]==ind:
            count=1
            dict1[w][i][1].append(ind1)
        if count==0:
          d7 = [ind1]
          dict1[w].append([ind,d7])

  return dict1

In [143]:
d1 = positional_inverted_index(doc10)

In [144]:
s1 = input("Input query: ")

s1 = re.sub(f"[{re.escape(punctuation)}]", "", s1)
s1 = s1.lower()
s1 = " ".join(s1.split())

s1

Input query: please let be


'please let be'

In [145]:
s1 = remove_stopwords(s1)
s1 = s1.split()
s1

['please', 'let']

performing the merging of postings for the query operations

In [146]:
def search(x,y,k):
    a=0
    b=0
    ans=[]
    while a<len(x) and b<len(y):
#         print(x[a][1])
        if x[a][0]==y[b][0]:
            
            try:
                p1 = np.array(x[a][1])
                p1 = list(p1)
            except:
                p1 = [x[a][1]]
                
            try:
                p2 = np.array(y[b][1])
                p2 = list(p2)
            except:
                p2 = [y[b][1]]
            c=0
#             print(p1)
#             print(p2)
            while c<len(p1):
                d=0
                l=[]
                while d<len(p2):
                    if abs(p1[c]-p2[d])<=k:
#                         print(p2[d])
                        l.append(p2[d])
                    elif p2[d]>p1[c]:
                        break
                    d+=1
                while (len(l)!=0 and abs(l[0]-p1[c]>k)):
                    l.pop(0)
#                 print(l)
#                 print(p1[c])
                for j in range(len(l)):
                    count=0
                    for i in range(len(ans)):
                        if(ans[i][0]==x[a][0]):
                            ans[i][1].append(p1[c])
                            ans[i][1].append(l[j])
                            count=1
                            break
                    if count==0:
                        ans.append([x[a][0],[p1[c],l[j]]])
                c+=1
            a+=1
            b+=1
#             print("ans")
#             print(ans)
        elif x[a][0]<y[b][0]:
            a+=1
        else:
            b+=1
                                        
    for i in range(len(ans)):
        ans[i][1] = np.unique(ans[i][1])
    return ans
        

Performing the query

In [147]:

temp = d1[s1[0]]
for j in range(1,len(s1)):
    ans1 = search(temp,d1[s1[j]],j)

    print(s1)
    
    temp = ans1
        

    
temp

['please', 'let']


[[0, array([40, 41])],
 [8, array([550, 551])],
 [57, array([40, 41])],
 [128, array([2053, 2054])],
 [188, array([304, 305])],
 [344, array([1892, 1893])],
 [1035, array([2319, 2320])]]

The documents name

In [148]:
for i in range(len(temp)):
  # print(temp[i][0])
  print(name[temp[i][0]])
  

1st_aid.txt
acne1.txt
arthriti.txt
bnb_quot.txt
carowner.txt
epi_.txt
top10st1.txt


No of documents retrieved

In [149]:
print("No. of documents : "+ str(len(temp)))

No. of documents : 7


In [150]:
name[8]

'acne1.txt'

In [151]:
len(name)

1133